In [1]:
import bempp.api
import numpy as np

bempp.api.global_parameters.quadrature.medium.double_order = 4
bempp.api.global_parameters.quadrature.far.double_order = 4

In [2]:
# Vértices do cubo
XYZ = np.array([[0, 0, 0],
                [1, 0, 0],
                [0, 1, 0],
                [1, 1, 0],
                [0, 0, 1],
                [1, 0, 1],
                [0, 1, 1],
                [1, 1, 1]])
# Conectividade do cubo
tri = np.array([[1,  4,  2],
                 [1,   3, 4],
                 [1,   6,  5],
                 [1,   2,  6],
                 [2,   8,   6],
                 [2,   4, 8],
                 [3,   8, 4],
                 [3,  7,  8],
                 [1,  7,   3],
                 [1,  5,  7],
                 [5,   8,  7],
                 [5,  6,  8]]) - 1

domain_index=np.array([1,1,2,2,3,3,4,4,5,5,6,6]) # Número das faces as quais os elementos pertencem

# Passa os dados da malha para o Bempp
grid= bempp.api.grid_from_element_data(XYZ.transpose(), tri.transpose(),domain_index) 

In [3]:
dirichlet_segments = [1,6] # Faces do cubo com condições de contorno de Dirichlet
neumann_segments = [2,3,4,5] # Faces do cubo com condições de contorno de Neumann

In [4]:
# Esta célula define as condições de contorno

# Esta function define o valor das condições de contorno de dirichlet nas faces onde o pontencial é conhecido
def dirichlet_data_fun(x,domain_index):
    if(domain_index==1): # Condição de contorno na face 1 (temperatura = 0)
        return 0
    else: # Condição de contorno na face 6 (temperatura = 1)
        return 1

# Esta function define o valor das condições de contorno de neumann nas faces onde o fluxo é conhecido
def neumann_data_fun(x):
    return 0 # Valor da CDC em todas as faces laterais do cubo (fluxo nulo)


In [5]:
order_neumann = 0 # Ordem das funções de forma que interpolam o fluxo
order_dirichlet = 1 # Ordem das funções de forma que interpolam o potencial

# O espaço no Bempp tem a malha triangular (grid), o tipo de elemento sendo contínuo = P (polynomial) e descontínuo
# igual a DP (discontinuous polinômio) e, por último, a ordem das funções de forma.

# define o tipo de elemento que interpola fluxo (triangular descontínuo de ordem order_neumann)
global_neumann_space = bempp.api.function_space(grid, "DP", order_neumann)

# define o tipo de elemento que interpola potencial (triangular contínuo de ordem order_dirichlet)
global_dirichlet_space = bempp.api.function_space(grid, "P", order_dirichlet)

# o restante desta célula eu tenho dúvidas (está confuso o significado para mim). Parece que está redundante com
# a próxima célula

# Espaço que armazena as variáveis de fluxo desconhecidas (range) no domínio (onde se aplica os pontos fontes) 
# das condições de contorno de Dirichlet (equação integral de potencial)
neumann_space_dirichlet_segment = bempp.api.function_space(
    grid, "DP", order_neumann, domains=dirichlet_segments,
    closed=True, element_on_segment=True)

# Espaço que armazena as variáveis de fluxo desconhecidas (range) no domínio (onde se aplica os pontos fontes) 
# das condições de contorno de Neumann
neumann_space_neumann_segment = bempp.api.function_space(
    grid, "DP", order_neumann, domains=neumann_segments,
    closed=False, element_on_segment=True, reference_point_on_segment=False)

# Espaço que armazena as variáveis de dirichlet desconhecidas (range) no domínio (onde se aplica os pontos fontes) 
# das condições de contorno de Dirichlet
dirichlet_space_dirichlet_segment = bempp.api.function_space(
    grid, "P", order_dirichlet, domains=dirichlet_segments, closed=True)

dirichlet_space_neumann_segment = bempp.api.function_space(
    grid, "P", order_dirichlet, domains=neumann_segments, closed=False)

dual_dirichlet_space = bempp.api.function_space(
    grid, "P", order_dirichlet, domains=dirichlet_segments,
    closed=True, strictly_on_segment=True)

In [7]:
# Estas são as 10 matrizes nas quais as matrizes H, G e suas derivadas são decompostas

# Equação de pontencial (regular), pontos fontes nos elementos onde o potencial é conhecido
# e elementos na região onde o potencial é conhecido. Slp = single layer potencial = matriz G
# Fica no lado esquerdo da equação matricial
slp_DD = bempp.api.operators.boundary.laplace.single_layer(
    neumann_space_dirichlet_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)  # used in "blocked"

# Equação de pontencial (regular), pontos fontes nos elementos onde o potencial é conhecido
# e elementos na região onde o fluxo é conhecido. dlp= double layer potencial = matriz H
# Fica no lado esquerdo da equação matricial
dlp_DN = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_space_neumann_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)  # used in "blocked"

# Equação de fluxo (hipersingular), pontos fontes nos elementos onde o fluxo é conhecido
# e elementos na região onde o potencial é conhecido. adlp = adjoint double layer potencial = derivada
# da matriz G (matriz D)= transposta da matriz H
# Fica no lado esquerdo da equação matricial
adlp_ND = bempp.api.operators.boundary.laplace.adjoint_double_layer(
    neumann_space_dirichlet_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)  # used in "blocked"


# Equação de fluxo (hipersingular), pontos fontes nos elementos onde o fluxo é conhecido
# e elementos na região onde o fluxo é conhecido. hyp = hipersingular = derivada da matriz H (matriz S)
# Fica no lado esquerdo da equação matricial
hyp_NN = bempp.api.operators.boundary.laplace.hypersingular(
    dirichlet_space_neumann_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)  # used in "blocked"


# Equação de pontencial (regular), pontos fontes nos elementos onde o potencial é conhecido
# e elementos na região onde o fluxo é conhecido. Slp = single layer potencial = matriz G
# Fica no lado direito da equação matricial
slp_DN = bempp.api.operators.boundary.laplace.single_layer(
    neumann_space_neumann_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)  # not used in "blocked"


# Equação de pontencial (regular), pontos fontes nos elementos onde o potencial é conhecido
# e elementos na região onde o potencial é conhecido. dlp= double layer potencial = matriz H
# Fica no lado direito da equação matricial
dlp_DD = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_space_dirichlet_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)  # not used in "blocked"

# Matriz de massa da equação de potencial, correpondente ao termo cij do método de colocação
# Fica no lado direito da equação matricial
id_DD = bempp.api.operators.boundary.sparse.identity(
    dirichlet_space_dirichlet_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)  # not used in "blocked"

# Equação de fluxo (hipersingular), pontos fontes nos elementos onde o fluxo é conhecido
# e elementos na região onde o fluxo é conhecido. adlp = adjoint double layer potencial = derivada
# da matriz G (matriz D) = transposta da matriz H
# Fica no lado direito da equação matricial
adlp_NN = bempp.api.operators.boundary.laplace.adjoint_double_layer(
    neumann_space_neumann_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)  # not used in "blocked"

# Matriz de massa da equação de fluxo, correpondente ao termo cij do método de colocação
# Fica no lado direito da equação matricial
id_NN = bempp.api.operators.boundary.sparse.identity(
    neumann_space_neumann_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)  # not used in "blocked"

# Equação de fluxo (hipersingular), pontos fontes nos elementos onde o fluxo é conhecido
# e elementos na região onde o potencial é conhecido. hyp = hipersingular = derivada da matriz H
# Fica no lado esquerdo da equação matricial
hyp_ND = bempp.api.operators.boundary.laplace.hypersingular(
    dirichlet_space_dirichlet_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)  # not used in "blocked"


# Junta as 4 matrizes que ficam no lado esquerdo da equação matricial em apenas uma matriz
blocked = bempp.api.BlockedOperator(2, 2)

blocked[0, 0] = slp_DD
blocked[0, 1] = -dlp_DN
blocked[1, 0] = adlp_ND
blocked[1, 1] = hyp_NN

In [8]:
# Passa as condições de contorno para o Bempp

def dirichlet_data(x, n, domain_index, res):
    res[0] = dirichlet_data_fun(x,domain_index)

def neumann_data(x, n, domain_index, res):
    res[0] = neumann_data_fun(x)

dirichlet_grid_fun = bempp.api.GridFunction(
    dirichlet_space_dirichlet_segment,
    fun=dirichlet_data, dual_space=dual_dirichlet_space)

neumann_grid_fun = bempp.api.GridFunction(
    neumann_space_neumann_segment,
    fun=neumann_data, dual_space=dirichlet_space_neumann_segment)


# Cria o dois vetores que compõe o lado direito da equação matricial. Só aqui os elementos 
# das matrizes do lado direito são efetivamente calculados (primeiro lugar onde as
# funções implementadas em C++ são usadas)
rhs_fun1 = (.5 * id_DD + dlp_DD) * dirichlet_grid_fun \
           - slp_DN * neumann_grid_fun
rhs_fun2 = - hyp_ND * dirichlet_grid_fun \
           + (.5 * id_NN - adlp_NN) * neumann_grid_fun

In [9]:
# Cria o lado esquerdo e direito da equação

lhs = blocked.weak_form() # Só agora os elementos das matrizes do lado esquerdo são calculados
rhs = np.hstack([rhs_fun1.projections(neumann_space_dirichlet_segment), 
                 rhs_fun2.projections(dirichlet_space_neumann_segment)])
from scipy.sparse.linalg import gmres
x, info = gmres(lhs, rhs)

In [10]:
# Separa o que é fluxo do que é potencial

nx0 = neumann_space_dirichlet_segment.global_dof_count

# os primeiros nx0 elementos do vetor x são fluxos
neumann_solution = bempp.api.GridFunction(
    neumann_space_dirichlet_segment, coefficients=x[:nx0])
# a partir do elementos nx0+1 do vetor x, são temperaturas calculadas
dirichlet_solution = bempp.api.GridFunction(
    dirichlet_space_neumann_segment, coefficients=x[nx0:])

In [11]:
# neste caso nx0 = 4, ou seja, todo o vetor x corresponde a fluxos desconhecidos
nx0

4

In [12]:
# Necessário para plotar gráfico no Jupyter
bempp.api.PLOT_BACKEND = "ipython_notebook"

In [13]:
# Pós-processamento

neumann_imbedding_dirichlet_segment = \
    bempp.api.operators.boundary.sparse.identity(
        neumann_space_dirichlet_segment,
        global_neumann_space,
        global_neumann_space)

neumann_imbedding_neumann_segment = \
    bempp.api.operators.boundary.sparse.identity(
        neumann_space_neumann_segment,
        global_neumann_space,
        global_neumann_space)

dirichlet_imbedding_dirichlet_segment = \
    bempp.api.operators.boundary.sparse.identity(
        dirichlet_space_dirichlet_segment,
        global_dirichlet_space,
        global_dirichlet_space)

dirichlet_imbedding_neumann_segment = \
    bempp.api.operators.boundary.sparse.identity(
        dirichlet_space_neumann_segment,
        global_dirichlet_space,
        global_dirichlet_space)

dirichlet = (dirichlet_imbedding_dirichlet_segment * dirichlet_grid_fun +
             dirichlet_imbedding_neumann_segment * dirichlet_solution)

neumann = (neumann_imbedding_neumann_segment * neumann_grid_fun +
           neumann_imbedding_dirichlet_segment * neumann_solution)

dirichlet.plot()

A Jupyter Widget

A Jupyter Widget

In [13]:
neumann.plot()

A Jupyter Widget

A Jupyter Widget

In [17]:
# Eu criei esta célula para saber os valores de cada elemento das matrizes
slp_DD_mat = bempp.api.as_matrix(slp_DD.weak_form())
# dlp_DN_mat = bempp.api.as_matrix(dlp_DN.weak_form())
adlp_ND_mat = bempp.api.as_matrix(adlp_ND.weak_form())
# hyp_NN_mat = bempp.api.as_matrix(hyp_NN.weak_form())
slp_DN_mat = bempp.api.as_matrix(slp_DN.weak_form())
dlp_DD_mat = bempp.api.as_matrix(dlp_DD.weak_form())
id_DD_mat = bempp.api.as_matrix(id_DD.weak_form())
adlp_NN_mat = bempp.api.as_matrix(adlp_NN.weak_form())
id_NN_mat = bempp.api.as_matrix(id_NN.weak_form())
hyp_ND_mat = bempp.api.as_matrix(hyp_ND.weak_form())

In [26]:
# Vou imprimir a matriz slp_DD_mat
print(slp_DD_mat)

[[ 0.07980854  0.03847765  0.01678227  0.01818665]
 [ 0.03847765  0.07980854  0.01818665  0.01678227]
 [ 0.01678227  0.01818665  0.07980854  0.03847765]
 [ 0.01818665  0.01678227  0.03847765  0.07980854]]


In [21]:
# Valore desconhecidos. Neste caso, todos são fluxos uma vez que todos os vértices do cubo tem
# potenciais conhecidos
print(x)

[-1.00052571 -1.00055716  1.00162618  1.00016268]


In [23]:
# Print out the number of elements
number_of_elements = grid.leaf_view.entity_count(0)
print("The grid has {0} elements.".format(number_of_elements))

The grid has 12 elements.


In [24]:
# Número de dofs de neumann (conhecidos e desconhecidos). Neste caso 12 pois são 12 elementos constantes
number_of_global_neumann_dofs = global_neumann_space.global_dof_count 
print("Number of global neumann dofs: {0}".format(number_of_global_neumann_dofs))

Number of global neumann dofs: 12


In [25]:
# Número de dofs de dirichlet (conhecidos e desconhecidos). Neste caso 8 pois são 8 vértices 
# sendo que cada vértice é um nó
number_of_global_dirichlet_dofs = global_dirichlet_space.global_dof_count
print("Number of global dirichlet dofs: {0}".format(number_of_global_dirichlet_dofs))

Number of global dirichlet dofs: 8


In [45]:
## Centroides dos elementos (onde o fluxo é calculado ou especificado)
print("Matrix of global Neumann interpolation points: {0}".format(global_neumann_space.global_dof_interpolation_points))

Matrix of global Neumann interpolation points: [[ 0.66666667  0.33333333  0.33333333  0.66666667  1.          1.
   0.66666667  0.33333333  0.          0.          0.33333333  0.66666667]
 [ 0.33333333  0.66666667  0.          0.          0.33333333  0.66666667
   1.          1.          0.66666667  0.33333333  0.66666667  0.33333333]
 [ 0.          0.          0.66666667  0.33333333  0.66666667  0.33333333
   0.33333333  0.66666667  0.33333333  0.66666667  1.          1.        ]]


In [46]:
# Vetores normais em cada um dos 12 elementos
print("Matrix of normal directions at Neumann interpolation points: {0}".format(global_neumann_space.global_dof_normals))

Matrix of normal directions at Neumann interpolation points: [[ 0.  0.  0.  0.  1.  1.  0.  0. -1. -1.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  1.  1.  0.  0.  0.  0.]
 [-1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]]


In [35]:
# Mostra os vértices e as áreas de um elemento específico, no caso o primeiro elemento
corners = elements[0].geometry.corners
area = elements[0].geometry.volume
print("Corners: {0}".format(corners))
print("Area: {0}".format(area))

Corners: [[ 0.  1.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
Area: 0.5


In [39]:
# Valores das temperaturas nos 8 vértices (calculadas e especificadas)
dirichlet.coefficients

array([  4.82705663e-17,   4.34435097e-17,  -2.17628945e-17,
         0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00])

Explanation for the number of dofs: 

    - Dirichlet dofs uses linear continuous element. So, it has one dof per vertice, i.e, 8 dofs. All of them are known.
    - Neumann dofs uses constant elements. So, it has one dof per element, i.e., 12 dofs, where 4 are unknowns.